In [1]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time 
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *
from analystcommunity.read_connection_data_warehouse import run_read_dwd_query

import random
import time
import datetime

import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

from procurement_lib import GoogleSheet

In [2]:
todays_date = datetime.datetime.today().strftime('%Y-%m-%d')

In [3]:
query = """
select
    competitor.competitor_name,
    site.identifier_value as site_code,
    quotation_date.full_date AS quotation_date,
    su.source_id,
    ROUND(AVG(cpp.product_selected_price),2)::float as price
from dpr_product_pricing.fact_collected_product_prices cpp
    inner join dpr_shared.dim_date quotation_date
        on cpp.dim_quotation_date = quotation_date.date_id
    inner join dpr_shared.dim_time quotation_time
        on cpp.dim_quotation_time = quotation_time.time_id
    inner join dpr_shared.dim_site site
        on cpp.dim_site = site.site_id
    inner join dpr_shared.dim_category cat
        on cpp.dim_category = cat.category_id
    inner join dpr_product_pricing.dim_product_outlier_type outlier_type
        on cpp.dim_outlier_type = outlier_type.outlier_type_id
    inner join dpr_product_pricing.dim_product_source_type source_type
        on cpp.dim_source_type = source_type.source_type_id
    inner join dpr_product_pricing.dim_product_competitor competitor
        on cpp.dim_competitor = competitor.competitor_id
    inner join dpr_product_pricing.dim_product_competitor_type competitor_type
        on(
            case
                when cpp.super_category = 'Fruver'
                    then competitor.product_competitor_type_id_fruver = competitor_type.competitor_type_id
                when cpp.super_category = 'Multicategoría'
                    then competitor.product_competitor_type_id_multicategoria = competitor_type.competitor_type_id
            end
        )
    inner join dpr_shared.dim_stock_unit su
        on cpp.dim_stock_unit = su.stock_unit_id
where quotation_date.full_date >= '2023-12-15'
    AND competitor.competitor_name NOT ILIKE '%cayena%'
    AND site.identifier_value IN ('SPO','CWB','VCP','BHZ')
GROUP BY 1,2,3,4
"""
df_zkkkkk = run_read_dwd_query(query)

df_zkkkkk = df_zkkkkk.dropna().reset_index(drop=True)
df_zkkkkk['lifetime'] = 8

In [4]:
# Ensure dataframe is sorted by 'quotation_date'
df_zkkkkk = df_zkkkkk.sort_values(by='quotation_date')

# Generate the required rows for missing dates
new_rows = []

for (competitor, source_id), group in df_zkkkkk.groupby(['competitor_name', 'source_id']):
    group = group.sort_values(by='quotation_date')
    last_known_price = None
    last_known_date = None
    lifetime = 8
    
    for current_index in range(len(group)):
        current_date = group.iloc[current_index]['quotation_date']
        price = group.iloc[current_index]['price']
        
        # If this is not the first iteration, fill in missing dates
        if last_known_date is not None:
            days_diff = (current_date - last_known_date).days
            if days_diff > 1:
                for j in range(1, min(days_diff, lifetime + 1)):
                    new_date = last_known_date + timedelta(days=j)
                    new_row = {
                        'site_code': group.iloc[current_index]['site_code'],
                        'quotation_date': new_date,
                        'competitor_name': competitor,
                        'source_id': source_id,
                        'price': last_known_price,
                        'lifetime': lifetime - j
                    }
                    new_rows.append(new_row)
                    
                    # Stop if we reach a new datapoint date
                    if new_date + timedelta(days=1) == current_date:
                        break
        
        # Update the last known values and reset lifetime
        last_known_price = price
        last_known_date = current_date
        lifetime = 8  # Reset lifetime

    # After processing all known dates for the group, continue generating rows until lifetime reaches 0
    while lifetime > 0:
        last_known_date += timedelta(days=1)
        new_row = {
            'site_code': group.iloc[-1]['site_code'],
            'quotation_date': last_known_date,
            'competitor_name': competitor,
            'source_id': source_id,
            'price': last_known_price,
            'lifetime': lifetime - 1
        }
        new_rows.append(new_row)
        lifetime -= 1

# Append new rows to the dataframe
df_zkkkkk = df_zkkkkk.append(new_rows, ignore_index=True)

# Sort the final dataframe
df_zkkkkk = df_zkkkkk.sort_values(by=['competitor_name', 'source_id', 'quotation_date'])
df_zkkkkk['replica'] = df_zkkkkk['lifetime'] == 8

In [5]:
df_bench = df_zkkkkk.copy()
df_bench['quotation_date'] = pd.to_datetime(df_bench['quotation_date'])

In [6]:
def q1(x):
    return x.quantile(0.25)

In [21]:
df_c = df_bench.groupby(['quotation_date',"source_id"]).agg(competitors=('competitor_name',lambda x: x.nunique()),
                                                            q0_bench=('price', np.min),
                                                            q1_bench=('price', q1)).reset_index()

In [22]:
query = """
WITH RECURSIVE calendar(calendar_date) AS (
  SELECT DATE_TRUNC('day', DATE(GETDATE()) - INTERVAL '300 day')
  UNION ALL
  SELECT calendar_date + INTERVAL '1 day'
  FROM calendar
  WHERE calendar_date BETWEEN DATE_TRUNC('day', DATE(GETDATE()) - INTERVAL '300 day') AND DATE(GETDATE() - 1) 
),

info AS (
SELECT
    DATE(coalesce(prices.last_modified_at, prices.created_at)) as created_at,
    --p.region_code as region,
    --pp.name as parent_product_name,
    pp.frida_id as sku_id,
    MIN(coalesce(tiers.tax_price, prices.tax_price)) as price, 
    min(coalesce(tiers.sale_price,prices.sale_price)) as net_price
    

FROM postgres_growth."growth_pricing.prices_history" prices
LEFT JOIN postgres_growth."growth_pricing.price_tiers_history" tiers ON prices.id = tiers.price_history_id
--LEFT JOIN postgres_growth."growth_pricing.price_discount_history" dct ON prices.id = dct.price_history_id
LEFT JOIN postgres_growth."growth_pricing.skus" skus ON prices.sku_id = skus.id
LEFT JOIN postgres_main_co."purchase_orders.products" p ON skus.sku_id = p.frida_id
LEFT JOIN postgres_main_co."purchase_orders.products" pp ON COALESCE(p.parent_id, p.id) = pp.id

WHERE DATE(prices.created_at) >= DATE_TRUNC('day', DATE(GETDATE()) - INTERVAL '300 day')
 AND p.region_code IN ('SPO','CWB','VCP','BHZ')
 AND p.deleted_at IS NULL
 AND prices.created_by NOT ILIKE '%CATALOG%'
 AND pp.product_category_id IN (5,6,7,8,9,10,13,18) -- 1 ES FRUVER
GROUP BY 1,2--,3,4
),

done AS (
SELECT
  DATE(c.calendar_date) AS quotation_date,
  --s.region,
  --s.parent_product_name,
  (s.sku_id)::int as source_id,
  LAG(i.price IGNORE NULLS) OVER (PARTITION BY s.sku_id ORDER BY c.calendar_date)::FLOAT AS precio,
  LAG(i.net_price IGNORE NULLS) OVER (PARTITION BY s.sku_id ORDER BY c.calendar_date)::FLOAT AS precio_neto

FROM calendar c
CROSS JOIN (SELECT DISTINCT sku_id FROM info) s
LEFT JOIN info i ON c.calendar_date = i.created_at-1 AND s.sku_id = i.sku_id
)

SELECT *
FROM done
WHERE precio IS NOT NULL
AND quotation_date >= '2024-01-01'
"""
df = read_connection_data_warehouse.runQuery(query)  
df['quotation_date'] = pd.to_datetime(df['quotation_date'])

In [9]:
query = """
SELECT 
su.source_id,
b.benchmark_date as quotation_date,
b.min_price::FLOAT AS min_bench

FROM dpr_product_pricing.obt_benchmark_product_prices b
INNER JOIN dpr_shared.dim_stock_unit su on b.stock_unit_id = su.stock_unit_id

WHERE su.source_id in {skus}
 AND b.benchmark_date >= '2024-01-01'
""".format(skus=tuple(df.source_id.unique()))

df_b = run_read_dwd_query(query)
df_b['quotation_date'] = pd.to_datetime(df_b['quotation_date'])

In [10]:
query = """
SELECT 
su.source_id,
DATE(to_date(c.dim_date_dtd::TEXT, 'YYYYMMDD') + INTERVAL '1 day') AS quotation_date,
(CASE WHEN c.dtd_cost_local = 0 THEN m.inventory_p_fin ELSE c.dtd_cost_local END)::FLOAT  AS costo

FROM dpr_cross_business.int_dtd_cost c 
INNER JOIN dpr_shared.dim_stock_unit su on c.dim_stock_unit = su.stock_unit_id
left join dpr_cross_business.fact_cross_business_insights m  on m.dim_stock_unit = su.stock_unit_id AND m.dim_date = c.dim_date_dtd
WHERE su.source_id in {skus}
 AND c.dim_date_dtd >= 20231231
 AND costo > 0
""".format(skus=tuple(df.source_id.unique()))

df_d = run_read_dwd_query(query)
df_d['quotation_date'] = pd.to_datetime(df_d['quotation_date'])

In [23]:
df = pd.merge(df, df_c, left_on=['quotation_date',"source_id"], right_on=['quotation_date',"source_id"], how='left')
df = pd.merge(df, df_b, left_on=['quotation_date',"source_id"], right_on=['quotation_date',"source_id"], how='left')
df = pd.merge(df, df_d, left_on=['quotation_date',"source_id"], right_on=['quotation_date',"source_id"], how='left')

In [24]:
df['mg'] = (1-df.costo/df.precio)
df['nmg'] = (1-df.costo/df.precio_neto)

df['comp_min'] = (1-df.costo/df.min_bench)
df['comp_q0'] = (1-df.costo/df.q0_bench)
df['comp_q1'] = (1-df.costo/df.q1_bench)

df['pi_min'] = df.precio/df.min_bench
df['npi_min'] = df.precio_neto/df.min_bench

df['pi_q0'] = df.precio/df.q0_bench
df['npi_q0'] = df.precio_neto/df.q0_bench

df['pi_q1'] = df.precio/df.q1_bench
df['npi_q1'] = df.precio_neto/df.q1_bench

In [25]:
query = """
SELECT
    s.identifier_value AS city,
    DATE(fs.order_submitted_date) AS quotation_date,
    --cat.parent_description AS cat,
    sup.source_id,
    --s.identifier_value || ' || ' || sup.card_description AS product_name,
    (SUM(fs.gmv_pxq_local)/4.75)::FLOAT AS gmv_usd
FROM dpr_sales.fact_sales                   fs
INNER JOIN dpr_shared.dim_site              s   ON s.site_id = fs.dim_site
INNER JOIN dpr_shared.dim_product           dp  ON dp.product_id = fs.dim_product
INNER JOIN dpr_shared.dim_category          cat ON cat.category_id = dp.category_id
INNER JOIN dpr_shared.dim_stock_unit        su  ON su.product_id = fs.dim_product
INNER JOIN dpr_shared.dim_stock_unit        sup  ON nvl(nullif(su.source_parent_id,0),su.source_id) = sup.source_id

WHERE 
    fs.gmv_enabled = TRUE
    AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
    AND fs.fb_order_status_id IN (1,6,7,8)
    AND fs.is_deleted = FALSE
    AND cat.super_category = 'Multicategoría'
    AND fs.dim_status = 1
    AND dp.is_slot = 'false'
    AND fs.gmv_pxq_local > 0
    AND s.identifier_value IN ('SPO','CWB','VCP','BHZ')
    AND DATE(fs.order_submitted_date) >= '2024-01-01'
GROUP BY 1,2,3--,4,5
"""
df_gmv = run_read_dwd_query(query)
df_gmv['quotation_date'] = pd.to_datetime(df_gmv['quotation_date'])

In [26]:
merge_df = pd.merge(df_gmv, df, left_on=['quotation_date',"source_id"], right_on=['quotation_date',"source_id"], how='left')

In [27]:
merge_df['year_week'] = merge_df['quotation_date'].dt.strftime('%Y-%U')

In [28]:
merge_df['gmv_mix'] = merge_df['gmv_usd'] / merge_df.groupby(['year_week', 'city'])['gmv_usd'].transform('sum')

In [43]:
# Define a function to apply the custom aggregation
def custom_ventas(group, column):
    return (group[column] * group['gmv_mix']).sum()
# Group by 'city' and 'year_week', aggregate multiple columns with custom aggregation
df_final = merge_df.groupby(['city', 'year_week']).apply(lambda group: pd.Series({
    'gmv_mix': group['gmv_mix'].sum(),  # Compute sum of gmv_mix directly
    'mg': custom_ventas(group, 'mg'),
    'nmg': custom_ventas(group, 'nmg'),
    'comp_min': custom_ventas(group, 'comp_min'),
    'comp_q0': custom_ventas(group, 'comp_q0'),
    'comp_q1': custom_ventas(group, 'comp_q1'),
    'pi_min': custom_ventas(group, 'pi_min'),
    'npi_min': custom_ventas(group, 'npi_min'),
    'pi_q0': custom_ventas(group, 'pi_q0'),
    'npi_q0': custom_ventas(group, 'npi_q0'),
    'pi_q1': custom_ventas(group, 'pi_q1'),
    'npi_q1': custom_ventas(group, 'npi_q1')
})).reset_index()

In [44]:
# Define a function to apply the custom aggregation
def custom_ventas2(group, column):
    return (group[column] * group['gmv_mix']).sum()/group['gmv_mix'].sum()
merge_df2 = merge_df.dropna(how='any')
# Group by 'city' and 'year_week', aggregate multiple columns with custom aggregation
df_final2 = merge_df2.groupby(['city', 'year_week']).apply(lambda group: pd.Series({
    'gmv_mix': group['gmv_mix'].sum(),  # Compute sum of gmv_mix directly
    'mg': custom_ventas(group, 'mg'),
    'nmg': custom_ventas(group, 'nmg'),
    'comp_min': custom_ventas(group, 'comp_min'),
    'comp_q0': custom_ventas(group, 'comp_q0'),
    'comp_q1': custom_ventas(group, 'comp_q1'),
    'pi_min': custom_ventas(group, 'pi_min'),
    'npi_min': custom_ventas(group, 'npi_min'),
    'pi_q0': custom_ventas(group, 'pi_q0'),
    'npi_q0': custom_ventas(group, 'npi_q0'),
    'pi_q1': custom_ventas(group, 'pi_q1'),
    'npi_q1': custom_ventas(group, 'npi_q1')
})).reset_index()

In [42]:
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import interact

# Assuming df_final2 is your DataFrame with 'year_week' as a categorical string
# Ensure 'year_week' is treated as categorical
df_final2['year_week'] = df_final2['year_week'].astype('category')

# Function to plot data for a single city with numerical variables
def plot_city_data(city):
    fig = px.line(df_final2[df_final2['city'] == city], x='year_week',
                  y=['gmv_mix', 'mg', 'nmg', 'comp_min', 'comp_q0', 'comp_q1',
                     'pi_min', 'npi_min', 'pi_q0', 'npi_q0', 'pi_q1', 'npi_q1'],
                  title=f'Data for {city}',
                  labels={'value': 'Values', 'variable': 'Variables', 'year_week': 'Year-Week'},
                  category_orders={'year_week': df_final2['year_week'].unique()})

    fig.update_layout(xaxis_title='Year-Week', yaxis_title='Values')

    fig.show()

# Get unique city names
city_names = pd.Series(df_final2.city.unique()).sort_values()

# Create dropdown widget
dropdown = widgets.Dropdown(options=city_names, description='Select City:')

# Interact function to update plot based on dropdown selection
interact(plot_city_data, city=dropdown)


interactive(children=(Dropdown(description='Select City:', options=('BHZ', 'CWB', 'SPO', 'VCP'), value='BHZ'),…

<function __main__.plot_city_data(city)>

In [31]:
df_final.to_excel("info_w.xlsx")

In [32]:
df_final2.to_excel("info_w2.xlsx")